# Importation des données

Ce fichier a pour but d'expliquer les méthodes d'importation des données. Ces scripts seront utilisé direcement dans les prochaines parties (avec des fichiers *.py* permettant de télécharger les données directement). Il s'agit donc ici de détailler la procédure d'importation.

## Présentation générale des données
a completer

In [1]:
import pandas as pd
import requests
from io import BytesIO, StringIO
from zipfile import ZipFile

## Données de 2023

In [2]:
print("début de l'importation du nombre de validations par jour S1 2023")
url = "https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/validations-reseau-ferre-nombre-validations-par-jour-1er-semestre/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
data_validation_23 = pd.read_csv(url, sep=";")
print("fin de l'importation du nombre de validations par jour S1 2023")

print("début de l'importation profils horaires par jour type S1 2023")
url = "https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/validations-reseau-ferre-profils-horaires-par-jour-type-1er-semestre/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
data_profils_horaires_23 = pd.read_csv(url, sep=";")
print("fin de l'importation profils horaires par jour type S1 2023")


début de l'importation du nombre de validations par jour S1 2023
fin de l'importation du nombre de validations par jour S1 2023
début de l'importation profils horaires par jour type S1 2023
fin de l'importation profils horaires par jour type S1 2023


In [3]:
data_validation_23.head()

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,lda,CATEGORIE_TITRE,NB_VALD
0,2023-06-26,800,853,568,MONTSOULT,67000,?,28
1,2023-06-26,800,853,568,MONTSOULT,67000,AMETHYSTE,9
2,2023-06-26,800,853,568,MONTSOULT,67000,IMAGINE R,388
3,2023-06-26,800,853,568,MONTSOULT,67000,TST,114
4,2023-06-26,800,853,595,NOINTEL MOURS,67227,?,8


In [4]:
data_profils_horaires_23.head()

,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,lda,CAT_JOUR,TRNC_HORR_60,pourc_validations
0,100,110,300,GALLIENI,71817,JOVS,9H-10H,8.12
1,100,110,300,GALLIENI,71817,SAHV,0H-1H,0.91
2,100,110,300,GALLIENI,71817,SAHV,14H-15H,7.37
3,100,110,300,GALLIENI,71817,SAHV,17H-18H,7.94
4,100,110,300,GALLIENI,71817,SAHV,23H-0H,1.46


## Données de 2015 à 2022

Les données de validation du réseau ferré de 2015 à 2022 sont accessibles depuis le fichier suivant : chaque ligne correspond à une année qui donne accès au téléchargement des données. Il suffit donc de télécharger ces données une à une, et d'extraire les fichiers dont nous avons besoin depuis les dossiers ZIP que nous décompressons dans des dossiers temporaire.

In [5]:
url = "https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/histo-validations-reseau-ferre/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
data_archives = pd.read_csv(url, sep=";")
data_archives.rename(columns=dict(zip(data_archives.columns, ["date", "lien"])), inplace=True)


In [6]:
data_archives.head()

,date,lien
0,2017,https://data.iledefrance-mobilites.fr/api/expl...
1,2019,https://data.iledefrance-mobilites.fr/api/expl...
2,2021,https://data.iledefrance-mobilites.fr/api/expl...
3,2022,https://data.iledefrance-mobilites.fr/api/expl...
4,2018,https://data.iledefrance-mobilites.fr/api/expl...


On affiche le nom des années et fichiers pour vérifier qu'ils sont bien importés et suivre l'avancée du téléchargement.

In [54]:
annees = [annee for annee in range(2015, 2023)]
table=["NB_S1", "PRO_S1", "NB_S2", "PRO_S2"]
validations_annees=pd.DataFrame()
profils_annees=pd.DataFrame()
for annee in annees:
    print(annee)
    url_zip = data_archives[data_archives['date'] == annee]['lien'].iloc[0]
    response = requests.get(url_zip)
    with ZipFile(BytesIO(response.content)) as zipfile:
        # On récupère la liste des noms de fichier dans le ZIP
        file_names = zipfile.namelist()
        i=0
        for file_name in file_names:
            if file_name.endswith('.zip'): print("fichier zip")
            else:                
                name=table[i]
                print(name)
                i+=1
                # Lecture du contenu du fichier
                with zipfile.open(file_name) as file:
                    if file_name.endswith('.csv') :
                        globals()[name] = pd.read_csv(file, sep=";")
                    elif annee==2022:
                        if name=="NB_S2" or name=="PRO_S2": globals()[name] = pd.read_csv(file, sep=";")
                        else: globals()[name] = pd.read_csv(file, sep='\t', header=0)
                    else:
                        globals()[name] = pd.read_csv(file, sep='\t', header=0)
    NB=pd.concat([NB_S1, NB_S2])
    NB['annee']=annee
    PRO=pd.concat([PRO_S1, PRO_S2])
    PRO['annee']=annee
    print(len(NB))
    print(len(PRO))
    validations_annees=pd.concat([validations_annees, NB])
    profils_annees=pd.concat([profils_annees, PRO])
    


2015
NB_S1
PRO_S1
NB_S2


/tmp/ipykernel_339/881613741.py:22: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[name] = pd.read_csv(file, sep=";")


PRO_S2
1534736
156760
2016
NB_S1
PRO_S1
NB_S2


/tmp/ipykernel_339/881613741.py:27: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[name] = pd.read_csv(file, sep='\t', header=0)


PRO_S2
1554133
164222


...

PRO_S2
fichier zip
fichier zip
fichier zip
fichier zip
fichier zip
1941505
161790
2021
NB_S1
PRO_S1
NB_S2
PRO_S2
2148300
163924
2022
NB_S1
PRO_S1
NB_S2


/tmp/ipykernel_339/881613741.py:25: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  else: globals()[name] = pd.read_csv(file, sep='\t', header=0)


PRO_S2


/tmp/ipykernel_339/881613741.py:24: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  if name=="NB_S2" or name=="PRO_S2": globals()[name] = pd.read_csv(file, sep=";")


2194281
168514


D'où vient l'erreur !?

In [55]:
validations_annees

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_REFA_LDA,CATEGORIE_TITRE,NB_VALD,annee,lda
0,01/01/2015,100,110,1,PORTE MAILLOT,71379,AMETHYSTE,117,2015,NaN
1,01/01/2015,100,110,1,PORTE MAILLOT,71379,AUTRE TITRE,37,2015,NaN
2,01/01/2015,100,110,1,PORTE MAILLOT,71379,FGT,172,2015,NaN
3,01/01/2015,100,110,1,PORTE MAILLOT,71379,IMAGINE R,605,2015,NaN
4,01/01/2015,100,110,1,PORTE MAILLOT,71379,NAVIGO,2039,2015,NaN
...,...,...,...,...,...,...,...,...,...,...
1105942,31/12/2022,810,802,905,VILLEPINTE,NaN,IMAGINE R,212,2022,73547.0
1105943,31/12/2022,810,802,905,VILLEPINTE,NaN,NAVIGO,614,2022,73547.0
1105944,31/12/2022,810,802,905,VILLEPINTE,NaN,NAVIGO JOUR,6,2022,73547.0
1105945,31/12/2022,810,802,905,VILLEPINTE,NaN,NON DEFINI,6,2022,73547.0


In [56]:
profils_annees

,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_REFA_LDA,CAT_JOUR,TRNC_HORR_60,pourc_validations,annee,lda
0,100,110,1,PORTE MAILLOT,71379,DIJFP,0H-1H,1.96,2015,NaN
1,100,110,1,PORTE MAILLOT,71379,DIJFP,10H-11H,4.01,2015,NaN
2,100,110,1,PORTE MAILLOT,71379,DIJFP,11H-12H,4.43,2015,NaN
3,100,110,1,PORTE MAILLOT,71379,DIJFP,12H-13H,5.36,2015,NaN
4,100,110,1,PORTE MAILLOT,71379,DIJFP,13H-14H,5.53,2015,NaN
...,...,...,...,...,...,...,...,...,...,...
85249,810,802,905,VILLEPINTE,NaN,SAVS,5H-6H,"4,25",2022,73547.0
85250,810,802,905,VILLEPINTE,NaN,SAVS,6H-7H,"7,22",2022,73547.0
85251,810,802,905,VILLEPINTE,NaN,SAVS,7H-8H,"6,79",2022,73547.0
85252,810,802,905,VILLEPINTE,NaN,SAVS,8H-9H,"6,39",2022,73547.0


## test d'importation avec le script

In [62]:
from donnees_IDFM import donnees_archives
data1,data2=donnees_archives()

2015


/home/onyxia/work/Frequentation-des-stations-de-TC-en-Ile-de-France/1- Importation des données/donnees_IDFM.py:61: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  with zipfile.open(file_name) as file:


2016


/home/onyxia/work/Frequentation-des-stations-de-TC-en-Ile-de-France/1- Importation des données/donnees_IDFM.py:66: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  else: globals()[name] = pd.read_csv(file, sep='\t', header=0)
/home/onyxia/work/Frequentation-des-stations-de-TC-en-Ile-de-France/1- Importation des données/donnees_IDFM.py:66: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  else: globals()[name] = pd.read_csv(file, sep='\t', header=0)


2017


KeyboardInterrupt: 

In [58]:
data1

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_REFA_LDA,CATEGORIE_TITRE,NB_VALD,annee,lda
0,01/01/2015,100,110,1,PORTE MAILLOT,71379,AMETHYSTE,117,2015,NaN
1,01/01/2015,100,110,1,PORTE MAILLOT,71379,AUTRE TITRE,37,2015,NaN
2,01/01/2015,100,110,1,PORTE MAILLOT,71379,FGT,172,2015,NaN
3,01/01/2015,100,110,1,PORTE MAILLOT,71379,IMAGINE R,605,2015,NaN
4,01/01/2015,100,110,1,PORTE MAILLOT,71379,NAVIGO,2039,2015,NaN
...,...,...,...,...,...,...,...,...,...,...
1105942,31/12/2022,810,802,905,VILLEPINTE,NaN,IMAGINE R,212,2022,73547.0
1105943,31/12/2022,810,802,905,VILLEPINTE,NaN,NAVIGO,614,2022,73547.0
1105944,31/12/2022,810,802,905,VILLEPINTE,NaN,NAVIGO JOUR,6,2022,73547.0
1105945,31/12/2022,810,802,905,VILLEPINTE,NaN,NON DEFINI,6,2022,73547.0


In [59]:
data2

,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_REFA_LDA,CAT_JOUR,TRNC_HORR_60,pourc_validations,annee,lda
0,100,110,1,PORTE MAILLOT,71379,DIJFP,0H-1H,1.96,2015,NaN
1,100,110,1,PORTE MAILLOT,71379,DIJFP,10H-11H,4.01,2015,NaN
2,100,110,1,PORTE MAILLOT,71379,DIJFP,11H-12H,4.43,2015,NaN
3,100,110,1,PORTE MAILLOT,71379,DIJFP,12H-13H,5.36,2015,NaN
4,100,110,1,PORTE MAILLOT,71379,DIJFP,13H-14H,5.53,2015,NaN
...,...,...,...,...,...,...,...,...,...,...
85249,810,802,905,VILLEPINTE,NaN,SAVS,5H-6H,"4,25",2022,73547.0
85250,810,802,905,VILLEPINTE,NaN,SAVS,6H-7H,"7,22",2022,73547.0
85251,810,802,905,VILLEPINTE,NaN,SAVS,7H-8H,"6,79",2022,73547.0
85252,810,802,905,VILLEPINTE,NaN,SAVS,8H-9H,"6,39",2022,73547.0
